# Clustering Crypto

In [24]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt



### Deliverable 1: Preprocessing the Data for PCA

In [25]:
# Load the crypto_data.csv dataset.
# YOUR CODE HERE
df = pd.read_csv('Resources/files/crypto_data.csv')
df = df.rename(columns = {'Unnamed: 0':'Crypto'})
df.set_index(df.columns[0], inplace=True )

In [26]:
# Keep all the cryptocurrencies that are being traded.
# YOUR CODE HERE
df=df.loc[df.IsTrading == True]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [27]:
# Keep all the cryptocurrencies that have a working algorithm.
# YOUR CODE HERE
df.dropna(subset=['Algorithm'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   IsTrading        1144 non-null   bool   
 3   ProofType        1144 non-null   object 
 4   TotalCoinsMined  685 non-null    float64
 5   TotalCoinSupply  1144 non-null   object 
dtypes: bool(1), float64(1), object(4)
memory usage: 54.7+ KB


In [28]:
# Remove the "IsTrading" column. 
# YOUR CODE HERE
del df['IsTrading']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1144 entries, 42 to XBC
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CoinName         1144 non-null   object 
 1   Algorithm        1144 non-null   object 
 2   ProofType        1144 non-null   object 
 3   TotalCoinsMined  685 non-null    float64
 4   TotalCoinSupply  1144 non-null   object 
dtypes: float64(1), object(4)
memory usage: 53.6+ KB


In [29]:
# Remove rows that have at least 1 null value.
# YOUR CODE HERE
df.dropna(how="any", inplace=True)
df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [30]:
# Keep the rows where coins are mined.
# YOUR CODE HERE

#Assumption: Keep the rows where coins are mined means TotalCoinsMined > 0
df = df[df["TotalCoinsMined"] > 0]
df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Crypto,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [31]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE
coinNames_df = df[['CoinName']]

coinNames_df

,CoinName
Crypto,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex


In [32]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
# YOUR CODE HERE
del df['CoinName']
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532 entries, 42 to XBC
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        532 non-null    object 
 1   ProofType        532 non-null    object 
 2   TotalCoinsMined  532 non-null    float64
 3   TotalCoinSupply  532 non-null    object 
dtypes: float64(1), object(3)
memory usage: 20.8+ KB


In [33]:
# Use get_dummies() to create variables for text features.
# YOUR CODE HERE
X = pd.get_dummies(df, columns=['Algorithm','ProofType'])
X.head(100)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Crypto,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XJO,3.919574e+07,45000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XST,3.309373e+07,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
ZCC,1.479902e+08,1000000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [34]:
# Standardize the data with StandardScaler().
X = StandardScaler().fit_transform(X)
X


array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [35]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)
crypto_pca


array([[-0.32868108,  1.06788033, -0.54926809],
       [-0.31203274,  1.06808861, -0.54969264],
       [ 2.30930425,  1.59804216, -0.64439583],
       ...,
       [ 0.34132593, -2.34878306,  0.40543029],
       [-0.16690415, -1.99589071,  0.54597043],
       [-0.28444274,  0.83354986, -0.24146254]])

In [36]:
# Create a DataFrame with the three principal components.
# YOUR CODE HERE
colNames = ['PC 1','PC 2','PC 3']
pcs_df = pd.DataFrame(crypto_pca, columns=colNames, index=df.index)
print(pcs_df)

            PC 1      PC 2      PC 3
Crypto                              
42     -0.328681  1.067880 -0.549268
404    -0.312033  1.068089 -0.549693
1337    2.309304  1.598042 -0.644396
BTC    -0.148389 -1.345853  0.187835
ETH    -0.148137 -2.031280  0.386306
...          ...       ...       ...
ZEPH    2.473452  0.789568 -0.100094
GAP    -0.326726  1.067763 -0.549296
BDX     0.341326 -2.348783  0.405430
ZEN    -0.166904 -1.995891  0.545970
XBC    -0.284443  0.833550 -0.241463

[532 rows x 3 columns]


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [37]:
# Create an elbow curve to find the best value for K.
kList = list(range(1, 11))
inertia = []
for i in kList:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

elbow_data = {"k":kList, "inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x = "k", y="inertia", xticks=kList, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=4`


In [38]:
# Initialize the K-Means model.
k = 4
km = KMeans(n_clusters=k, random_state=0)

# Fit the model
km.fit(pcs_df)


# Predict clusters
predictions = km.predict(pcs_df)
predictions

array([3, 3, 3, 0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 0, 3, 3, 0, 0,
       0, 0, 0, 3, 0, 0, 0, 3, 0, 3, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3,
       0, 0, 0, 0, 0, 3, 3, 0, 3, 0, 0, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0,
       0, 0, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 0, 3, 0, 3, 3, 0, 0, 0, 0, 3,
       3, 0, 3, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0,
       3, 0, 3, 3, 0, 0, 3, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 3,
       0, 3, 0, 0, 3, 0, 3, 0, 3, 3, 0, 0, 3, 0, 0, 3, 3, 0, 3, 0, 3, 3,
       3, 0, 0, 0, 0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 3, 0, 3, 0, 3, 0, 3, 0, 3, 3, 3, 3,
       0, 3, 3, 3, 3, 3, 0, 0, 3, 3, 0, 0, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 0, 0, 0, 3, 3, 3, 3, 0, 3, 0, 3,
       3, 0, 3, 0, 0, 3, 0, 0, 3, 0, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 0, 3, 3, 3, 3, 0, 3, 0, 3, 0, 0, 0, 0, 3, 0, 3, 3, 0,
       3, 0, 0, 0, 3, 0, 3, 0, 0, 0, 3, 0, 3, 0, 3,

In [39]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis=1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = coinNames_df

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions


# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
Crypto,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.328681,1.067880,-0.549268,42 Coin,3
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.312033,1.068089,-0.549693,404Coin,3
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.309304,1.598042,-0.644396,EliteCoin,3
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148389,-1.345853,0.187835,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.148137,-2.031280,0.386306,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.159614,-1.069265,0.002099,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.406950,1.206874,-0.536690,Dash,3
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.150778,-2.244551,0.375705,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.146578,-2.031381,0.386285,Ethereum Classic,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [40]:
# Creating a 3D-Scatter with the PCA data and the clusters
# Plotting the clusters with three features

CoinNames = list(clustered_df["CoinName"])
Algorithms = list(clustered_df['Algorithm'])

fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="Class", symbol="Class",width=800, hover_data = ['Algorithm'], hover_name = "CoinName" )
fig.update_layout(legend=dict(x=0,y=1))
fig.show()


In [41]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinSupply","TotalCoinsMined","Class"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [42]:
# Print the total number of tradable cryptocurrencies.
print(f" Total Number of Tradable Cryptocurriences = {len(clustered_df)} ")

 Total Number of Tradable Cryptocurriences = 532 


In [43]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
mScale=MinMaxScaler()
plot_data=mScale.fit_transform(clustered_df[["TotalCoinSupply", "TotalCoinsMined"]])

##########################################################################################



In [44]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df = pd.DataFrame(
    plot_data, index=clustered_df.index, columns=["TotalCoinSupply", "TotalCoinsMined"])



plot_df['CoinName']=clustered_df['CoinName']
plot_df['Class']=clustered_df['Class']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
Crypto,,,,
42,4.200000e-11,0.000000,42 Coin,3
404,5.320000e-04,0.001066,404Coin,3
1337,3.141593e-01,0.029576,EliteCoin,3
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,3
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0


In [45]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
plot_df.hvplot.scatter(
    x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"], by="Class")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

In [46]:

plt.show()